<a href="https://colab.research.google.com/github/sabamisoon/AutoFX/blob/master/AutoFX_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install oandapyV20

In [0]:
from oandapyV20 import API
accountID = "***************"
access_token = "************************"
api = API(access_token=access_token ,environment="practice")

In [0]:
import json
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream

instruments = "EUR_USD,EUR_JPY"
s = PricingStream(accountID=accountID, params={"instruments":instruments})
MAXREC = 10
try:
    n = 0
    for R in api.request(s):
        print(json.dumps(R, indent=2))
        n += 1
        if n > MAXREC:
            s.terminate("maxrecs received {}".format(MAXREC)) 
except V20Error as e:
    print("Error: {}".format(e))

In [0]:
import json
import pandas as pd
import oandapyV20.endpoints.instruments as instruments
from oandapyV20.types import DateTime
from datetime import datetime ,timedelta
from dateutil.relativedelta import relativedelta

def get_equity_df( since ,until ,interval ,price ) :
    cnt = 'mid' if price=='M' else 'ask' if price=='A' else 'bid' if price=='B' else 'mid'
    params = { "from": DateTime( since ).value
              ,"to": DateTime( until ).value
              ,"price": price
              ,"granularity": interval }
    r = instruments.InstrumentsCandles( instrument="USD_JPY" ,params=params )
    api.request( r )    
    raw_list = []
    for raw in r.response['candles'] :
        raw_list.append( [ raw['time'] ,raw[cnt]['o'] ,raw[cnt]['h'] ,raw[cnt]['l'] ,raw[cnt]['c'] ,raw['volume'] ] )
    raw_df = pd.DataFrame( raw_list ,columns=[ 'Time' ,f'Open_{cnt}' ,f'High_{cnt}' ,f'Low_{cnt}' ,f'Close_{cnt}' ,'Volume' ] )
    return raw_df


start    = datetime.strptime( '2005-01-01' ,'%Y-%m-%d' )
end      = datetime.strptime( '2019-12-31' ,'%Y-%m-%d' )
month    = 12 * 2 #上限5000レコード
interval = 'H4'
restart  = 0

since = start + relativedelta( months=( restart ) )
until = start + relativedelta( months=( month + restart ) )
df = pd.DataFrame()

while True :
    if until > datetime.now() :
        until = datetime.now()
        if relativedelta( since ,until ).months==0 : break
    
    print( f'since:{ since } ,until:{ until }' )

    raw_a = get_equity_df( since ,until ,interval ,'A' )
    raw_b = get_equity_df( since ,until ,interval ,'B' )
    raw = pd.merge( raw_a ,raw_b )
    raw = raw.set_index( 'Time' )
    raw.index = pd.to_datetime( raw.index )

    df = pd.concat( [ df ,raw ] )

    since = until
    until = until + relativedelta( months=month )
    if since >= end : break

In [0]:
import oandapyV20.endpoints.orders as orders
order_data = {
        "order": {
          "instrument": "USD_JPY",
            "units": "+100",
            "type": "MARKET",
          }
      }
o = orders.OrderCreate(accountID, data=order_data)
api.request( o )
o.response

In [0]:
import oandapyV20.endpoints.orders as orders
order_data = {
        "order": {
           "price": "107.765"
          ,"instrument": "USD_JPY"
          ,"units": "+10000"
          ,"type": "STOP"
          }
      }
o = orders.OrderCreate(accountID, data=order_data)
api.request( o )
o.response

In [0]:
import oandapyV20.endpoints.orders as orders
c = orders.OrdersPending( accountID )
api.request( c )
c.response

In [0]:
import oandapyV20.endpoints.orders as orders
c = orders.OrderDetails( accountID ,orderID='4' )
api.request( c )
c.response

In [0]:
import oandapyV20.endpoints.positions as positions
position_data = { "longUnits": "ALL" }
p = positions.PositionClose( accountID=accountID, data=position_data, instrument="USD_JPY" )
api.request( p )
p.response

In [0]:
import oandapyV20.endpoints.positions as positions
p = positions.PositionDetails(accountID=accountID, instrument="USD_JPY")
api.request( p )
p.respons